In [1]:
## This one appears to be the most recent.  So...let's make sure it's working.

In [ ]:
# https://github.com/helgefmi/ohno/blob/master/ohno/client/pty.py ## loosely based on this code
import os
import pty
import subprocess
import pprint
import time
import json
import select
import re

ansi_escape_codes = {
    r"\x1b7": "save cursor",
    r"\x1b\[\?47h": "set alternate screen",
    r"\x1b\)0": "set to ASCII",
    r"\x1b\[H": "move cursor to upper left",
    r"\x1b\[2J": "clear screen", # this might be a parameter?
    r"\x1b\[(\d+);(\d+)H": "move cursor",
    r"\x1b\[K": "clear to end of line",
    r"\x1b\[C": "move cursor right",
    r"\x1b8": "restore cursor",
}

class Terminal:
    unsupported = {
        r"\x1b7": "save cursor",
        r"\x1b8": "restore cursor",
        r"\x1b\[\?47h": "set alternate screen",
        r"\x1b\)0": "set to ASCII",
    }

    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.reset()

    def reset(self):
        self.cursor_row = 0
        self.cursor_col = 0
        self.characters = [[' ' for i in range(self.cols)] for _ in range(self.rows)]
        self.foregrounds = [["white" for i in range(self.cols)] for _ in range(self.rows)]
        self.backgrounds = [["black" for i in range(self.cols)] for _ in range(self.rows)] # does Nethack actually do this?

    def text(self):
        s = ""
        for i in range(self.rows):
            for j in range(self.cols): # let's ignore colors for now
                s += self.characters[i][j]
            s += "\n"
        return s
    
    def print(self):
        print(self.text())
                
    def update(self, input, print_instructions = False):
        #self.reset() # I think this is how I left the code, so I think it's not supposed to reset
        # alright, we're gonna do this without stransi

        for instr in ansi.instructions():
            if print_instructions: 
                print(instr) # for now
            if isinstance(instr, str):
                for i in range(len(instr)):
                    ch = instr[i]
                    if ch == "\r": # carriage return; reset cursor column to 0
                        self.cursor_col = 0
                    elif ch == "\b": # backspace; move cursor column back by one
                        self.cursor_col -= 1
                    elif ch == "\n": # newline; move cursor row down by one
                        self.cursor_row += 1
                    elif ch == '\x0f' or ch == '\x00': # these should indeed be no-ops but I'm not sure why they're here
                        pass
                    else:
                        try:
                            self.characters[self.cursor_row][self.cursor_col] = ch
                            self.cursor_col += 1
                        except:
                            print(f"something out of bounds: col={self.cursor_col}, row={self.cursor_row}, i={i} len={len(instr)}")
            elif isinstance(instr, stransi.SetCursor):
                if instr.move.relative: # Move the cursor relative to its current position
                    self.cursor_row += instr.move.x
                    self.cursor_col += instr.move.y
                else: # Move the cursor to an absolute position
                    self.cursor_row = instr.move.x
                    self.cursor_col = instr.move.y 
            elif isinstance(instr, stransi.SetClear):
                if instr.region == stransi.clear.Clear.SCREEN:
                    self.reset()
                elif instr.region == stransi.clear.Clear.LINE_AFTER:
                    for i in range(self.cursor_col, self.cols):
                        self.characters[self.cursor_row][i] = ' ' # clear the rest of the line
                elif instr.region == stransi.clear.Clear.LINE:
                    print("stransi.clear.Clear.LINE, was not expecting this.")
                    for i in range(self.cols):
                        self.characters[self.cursor_row][i] = ' '
                elif instr.region == stransi.clear.Clear.LINE_BEFORE:
                    print("stransi.clear.Clear.LINE_BEFORE, was not expecting this.")
                    for i in range(0, self.cursor_col):
                        self.characters[self.cursor_row][i] = ' '
                elif instr.region == stransi.clear.Clear.SCREEN_AFTER:
                    print("stransi.clear.Clear.SCREEN_AFTER, was not expecting this.")
                    for i in range(self.cursor_row, self.rows):
                        for j in range(self.cols):
                            self.characters[i][j] = ' '
                elif instr.region == stransi.clear.Clear.SCREEN_BEFORE:
                    print("stransi.clear.Clear.SCREEN_BEFORE, was not expecting this.")
                    for i in range(0, self.cursor_row):
                        for j in range(self.cols):
                            self.characters[i][j] = ' '
                else:
                    print(instr)
                    print(instr.region)
                    print("I guess I didn't handle all the clears")
            elif isinstance(instr, stransi.SetColor):
                if instr.role == stransi.color.ColorRole.FOREGROUND:
                    self.foregrounds[self.cursor_row][self.cursor_col] = instr.color.ansi256.code
                elif instr.role == stransi.color.ColorRole.BACKGROUND:
                    self.backgrounds[self.cursor_row][self.cursor_col] = instr.color.ansi256.code
                else:
                    print("some other kind of color instruction?")
                    print(instr)
            elif isinstance(instr, stransi.SetAttribute):
                if instr.attribute == stransi.attribute.Attribute.NORMAL:
                    pass
                    #print("normal")
                elif instr.attribute == stransi.attribute.Attribute.BOLD:
                    pass
                    #print("bold")
                elif instr.attribute == stransi.attribute.Attribute.REVERSE:
                    pass
                    #print("reverse")
                else:
                    print("another attribute?")
                    print(instr)
            else:
                if print_instructions:
                    print("^^^ this instruction was not handled.")
        # this is how the version I'm drawing from did it
        if self.cursor_col >= self.cols:
            self.cursor_col = self.cursor_col - self.cols # wrap around to the next line
            self.cursor_row += 1
        if self.cursor_row >= self.rows:
            self.cursor_row = self.rows - 1 # don't go off the screen


class Nethack:
    def __init__(self):
        self.master = None
        self.slave = None
        self.process = None
        self.terminal = Terminal(24, 80)

    def start_process(self):
        self.master, self.slave = pty.openpty()
        self.process = subprocess.Popen(
            ["nethack"],
            stdin=self.slave,
            stdout=self.slave,
            stderr=self.slave,
            close_fds=True, # this line comes from Bing Chat and I don't know what it does really
            universal_newlines=True,
        )
        os.close(self.slave) # Copilot added this so take it with a grain of salt...if it works, we don't need to save the reference

    def read_output(self, timeout = 1.0, sleep = 0.25, chars = 8192):
        if self.process is None:
            raise ValueError("Process not started")
        if sleep is not None and sleep > 0:
            time.sleep(sleep)
        if timeout is not None and timeout > 0:
            rlist, _, _ = select.select([self.master], [], [], timeout)
            if rlist:
                return os.read(self.master, chars).decode()
            else:
                print("(read timed out)")
                print("(this usually means the input sent did not update the game state.)")
                return None
        else:
            return os.read(self.master, chars).decode()

    def send_command(self, command):
        #os.write(self.master, command.encode() + b"\n")
        os.write(self.master, command.encode())

    def close(self):
        try:
            self.process.terminate()
            os.close(self.master)
        except:
            pass

    def next_command(self, c, verbose = False, timeout = 1.0):
        self.send_command(c)
        text = self.read_output(timeout = timeout)
        if verbose:
            print(text)
        if text is not None:
            self.terminal.update(text, print_instructions = verbose)
        print(self.terminal.text())
        return text

    @staticmethod
    def clean_locks():
        password = json.load(open("private.json"))["sudo"]
        os.system(f'echo "{password}" | sudo -S rm -f /var/games/nethack/*lock*')

    @staticmethod
    def get_new_process():
        Nethack.clean_locks()
        n = Nethack()
        n.start_process()
        text = n.read_output(timeout = 1.0)
        n.terminal.update(text)
        print(n.terminal.text())
        return n

# 24, 80

In [3]:
nethack = Nethack.get_new_process()

[sudo] password for perplexity: 

found unsupported \x1b7 : save cursor
found unsupported \x1b\[\?47h : set alternate screen
found unsupported \x1b\)0 : set to ASCII
Restoring save file...--More--                                                  
NetHack, Copyright 1985-2023                                                    
         By Stichting Mathematisch Centrum and M. Stephenson.                   
         Version 3.6.7 Unix, revised Apr  1 07:02:11 2024.                      
         See license for details.                                               
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                          

In [4]:
nethack.next_command("n")

(read timed out)
(this usually means the input sent did not update the game state.)
Restoring save file...--More--                                                  
NetHack, Copyright 1985-2023                                                    
         By Stichting Mathematisch Centrum and M. Stephenson.                   
         Version 3.6.7 Unix, revised Apr  1 07:02:11 2024.                      
         See license for details.                                               
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                         

In [5]:
nethack.next_command("v")

(read timed out)
(this usually means the input sent did not update the game state.)
Restoring save file...--More--                                                  
NetHack, Copyright 1985-2023                                                    
         By Stichting Mathematisch Centrum and M. Stephenson.                   
         Version 3.6.7 Unix, revised Apr  1 07:02:11 2024.                      
         See license for details.                                               
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                         

In [6]:
nethack.next_command("h")

(read timed out)
(this usually means the input sent did not update the game state.)
Restoring save file...--More--                                                  
NetHack, Copyright 1985-2023                                                    
         By Stichting Mathematisch Centrum and M. Stephenson.                   
         Version 3.6.7 Unix, revised Apr  1 07:02:11 2024.                      
         See license for details.                                               
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                         

In [7]:
nethack.next_command("n")

(read timed out)
(this usually means the input sent did not update the game state.)
Restoring save file...--More--                                                  
NetHack, Copyright 1985-2023                                                    
         By Stichting Mathematisch Centrum and M. Stephenson.                   
         Version 3.6.7 Unix, revised Apr  1 07:02:11 2024.                      
         See license for details.                                               
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                         

In [8]:
nethack.next_command("y")

(read timed out)
(this usually means the input sent did not update the game state.)
Restoring save file...--More--                                                  
NetHack, Copyright 1985-2023                                                    
         By Stichting Mathematisch Centrum and M. Stephenson.                   
         Version 3.6.7 Unix, revised Apr  1 07:02:11 2024.                      
         See license for details.                                               
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                         

In [9]:
nethack.next_command(" ")

Velkommen perplexity, the human Valkyrie, welcome back to NetHack!--More--      
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                            

'\x1b[H\x1b[K\x1b[H\x1b[2J\x1b[H\x1b[14;24H----------.-----\x1b[m\x1b[15;24H|..............|\x1b[m\x1b[16;24H|...\x1b[m\x1b\x1b%\x1bd\x1b[m\x1b[m.........|\x1b[m\x1b[17;24H|.....\x1b[m\x1b\x1b@\x1b[m.........\x1b[m\x1b[18;24H----------------\x1b[m\x1b[17;30H\x1b[23;1H\x1b[K\x1b[24;1H\x1b[K\x1b[A[\x1bPerplexity the Stripling      \x1b[m] St:18/01 Dx:11 Co:18 In:10 Wi:8 Ch:9 Neutral\x1b[K\r\x1b[24;1HDlvl:1 $:0 HP:16(16) Pw:2(2) AC:6 Xp:1\x1b[K\x1b[HVelkommen perplexity, the human Valkyrie, welcome back to NetHack!\x1b[K\x1b[17;30H\x1b[1;67H--More--'

In [10]:
# h is left, j is down, k is up, l is right
#text = next_command("j")  # okay, something odd is happening here: When I move, the player cursor disappears, and then the right way moves to the left.  So it seems like some portion of the text is getting blotted out.
# interesting, that final character is \x08 which is backspace.  I wonder if that has smething to do with it.
nethack.next_command("l", timeout = 0.5)
# The problems I'm seeing here seem to be of a form where the I see the new locations of entities, but the old locations are not being cleared.
# It's actually a bit weirder than that.  There's some jumping around.  That kind of makes it seem like some of the control characters are not being handled correctly.
# Which sucks, because I know I already put a ton of work into that.  And I know it's a pain to debug.
# So, one question here is whether the things are not updating, or whether they're updating incorrectly.  That should actually not be too hard to figure out; we could just 
# clear the screen either to blanks or to a single character, and then see if the entities are being drawn correctly.
# So it looks to be a matter of not updating things that should be updating; however, we're also getting some strange things...
# ...the player did not get drawn after switching places with the pet.
# ...sometimes the player "teleports" to the top-center of the screen.
# When I get rid of the full clear, we do see some teleporting around.  You might expect that if the screen is getting re-centered, but then you would expect other things to happen as well.

(read timed out)
(this usually means the input sent did not update the game state.)
Velkommen perplexity, the human Valkyrie, welcome back to NetHack!--More--      
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                         